In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob
import pathlib
import statistics
%matplotlib inline

In [3]:
class ColorReduction:
    def __call__(self, img):
        if len(img.shape) == 3:
            return self.apply_3(img)
        if len(img.shape) == 2:
            return self.apply_2(img)
        return None
        
    # problem 84 の reference solution は、ここの処理間違ってそう
    def reduction_onepixel(self, value):
        if 0 <= value < 64:
            return 32
        elif 64 <= value < 128:
            return 96
        elif 128 <= value < 192:
            return 160
        elif 192 <= value < 256:
            return 224
        return -1
    
    def apply_3(self, img):
        H, W, ch = img.shape
        output_img = img.copy()
        for i in range(H):
            for j in range(W):
                for c in range(ch):
                    output_img[i, j, c] = self.reduction_onepixel(img[i, j, c])
        return output_img
    
    def apply_2(self, img):
        H, W = img.shape
        output_img = img.copy()
        for i in range(H):
            for j in range(W):
                output_img[i, j] = self.reduction_onepixel(img[i, j])
        return output_img

In [26]:
class KmeansImageRecognition:
    def __init__(self, parse_func, class_list):
        self.color_reduction = ColorReduction()
        self.reduced_valuemap = {
            32: 0,
            96: 1,
            160: 2,
            224: 3
        }
        self.parse_func = parse_func
        self.class_list = class_list
        self.num_classes = len(class_list)
         
    def _get_images(self, test_path):
        images, names = [], []
        file_list = sorted(glob.glob(test_path))
        for file in file_list:
            images.append(cv2.imread(file))
            names.append(file)
        images = np.array(images)
        names = np.array(names)
        return images, names
    
    def _get_hist(self, img):
        assert len(img.shape) == 3, "invalid img dimension: expected: 3, got: {}".format(img.shape)
        H, W, ch = img.shape
        
        hist = np.zeros((12))
        for i in range(H):
            for j in range(W):
                for c in range(ch):
                    cls = 4*c + self.reduced_valuemap[self.color_reduction.reduction_onepixel(img[i, j, c])]
                    hist[cls] += 1
        return hist
    
    def _get_hists(self, images):
        # create histograms
        hists = np.zeros((len(images), 12))
        for i in range(len(images)):
            hists[i] = self._get_hist(images[i])
        return hists
    
    def partial_kmeans(self, test_path, th=0.5):
        images, names = self._get_images(test_path)
        hists = self._get_hists(images)
        N = len(images)
        classes = np.array([0 if np.random.random() < th else 1 for i in range(N)])
        
        # 何も属さないようなクラスは存在しないと仮定
        gs = np.zeros((self.num_classes, 12), dtype=np.float32)
        ns = np.zeros((self.num_classes))
        ns[0] = len(classes == 0)
        ns[1] = len(classes == 1)
        gs[0] = np.sum(hists[classes == 0], axis=0).astype(np.float32) / ns[0]
        gs[1] = np.sum(hists[classes == 1], axis=0).astype(np.float32) / ns[1]
        print("assigned: {}".format(classes))
        print(gs[0])
        print(gs[1])
        
    def problem_88(self, test_path, th=0.5):
        self.partial_kmeans(test_path)

In [27]:
def parse_func(file_name):
    return file_name.split("_")[1]
class_list = np.array(["akahara", "madara"])

recog = KmeansImageRecognition(parse_func, class_list)
recog.problem_88("../dataset/test_*.jpg", 0.5)

assigned: [0 0 0 1]
[2026.5  6074.   2188.5  1999.   2006.   6479.5  2290.   1512.5  1908.25
 5751.   2910.   1718.75]
[ 852.5  2090.   1123.5    30.    967.25 1909.25  982.25  237.25  918.25
 2137.25  985.     55.5 ]
